<a href="https://colab.research.google.com/github/fran1215/MLAI/blob/main/W6_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04

/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04


In [17]:
import numpy as np
import pandas as pd

weather = pd.read_csv('data/nyc_weather_2018.csv', parse_dates=['date'])
weather['date'] = pd.to_datetime(weather['date'])
df = weather.set_index('date')
df = df[df.datatype=='PRCP']

In [19]:
df

,datatype,station,attributes,value
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,",,N,",0.0
2018-01-01,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2018-01-01,PRCP,GHCND:US1NJBG0017,",,N,",0.0
2018-01-01,PRCP,GHCND:US1NJBG0018,",,N,",0.0
2018-01-01,PRCP,GHCND:US1NJBG0023,",,N,",0.0
...,...,...,...,...
2018-12-31,PRCP,GHCND:USW00054787,",,W,",28.7
2018-12-31,PRCP,GHCND:USW00094728,",,W,2400",25.9
2018-12-31,PRCP,GHCND:USW00094741,",,W,",29.2


In [25]:
dp = df.groupby(level=0).sum()
dp

,value
date,
2018-01-01,0.0
2018-01-02,0.0
2018-01-03,0.0
2018-01-04,273.8
2018-01-05,649.4
...,...
2018-12-27,0.0
2018-12-28,853.9
2018-12-29,1430.6


In [24]:
dp.groupby(pd.Grouper(freq='Q')).sum()

,value
date,
2018-03-31,26724.5
2018-06-30,26174.6
2018-09-30,43181.7
2018-12-31,37768.5


In [26]:
dp.groupby(pd.Grouper(freq='Q')).value.count()

date
2018-03-31    90
2018-06-30    91
2018-09-30    92
2018-12-31    92
Freq: Q-DEC, Name: value, dtype: int64

In [29]:
dp.assign(
    total_prcp_in_q=lambda x: \
      x.groupby(pd.Grouper(freq='Q')).transform(np.sum),
    total_day_count_in_q=lambda x: \
      x.groupby(pd.Grouper(freq='Q')).value.transform('count'),
    pct_monthly_prcp=lambda x: \
      x.value.div(x.total_prcp_in_q)
)

,value,total_prcp_in_q,total_day_count_in_q,pct_monthly_prcp
date,,,,
2018-01-01,0.0,26724.5,90,0.000000
2018-01-02,0.0,26724.5,90,0.000000
2018-01-03,0.0,26724.5,90,0.000000
2018-01-04,273.8,26724.5,90,0.010245
2018-01-05,649.4,26724.5,90,0.024300
...,...,...,...,...
2018-12-27,0.0,37768.5,92,0.000000
2018-12-28,853.9,37768.5,92,0.022609
2018-12-29,1430.6,37768.5,92,0.037878


Task 2

In [45]:
fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903,low
2018-01-03,181.88,184.78,181.3300,184.67,16886563,low
2018-01-04,184.90,186.21,184.0996,184.33,13880896,low
2018-01-05,185.59,186.90,184.9300,186.85,13574535,low
2018-01-08,187.20,188.90,186.3300,188.28,17994726,low


In [47]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month']
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0


In [52]:
fb.pivot_table(
    index='trading_volume', columns=fb.index.month, values='close', aggfunc='count'
    )

date,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0
